# GitHub Repository Update

This notebook updates the RaspberryPi-CAN repository from GitHub with proper error handling for offline scenarios.

**Just run the cell below to update the repository!**

In [ ]:
import os
import socket
import subprocess
from datetime import datetime


def check_internet_connection(host='8.8.8.8', port=53, timeout=5):
    """Check if internet connection is available."""
    try:
        with socket.create_connection((host, port), timeout=timeout):
            return True
    except OSError:
        return False


def run_command(args, cwd=None, timeout=120):
    """Run command and return (success, stdout, stderr)."""
    env = os.environ.copy()
    env['GIT_TERMINAL_PROMPT'] = '0'
    try:
        result = subprocess.run(
            args,
            cwd=cwd,
            capture_output=True,
            text=True,
            timeout=timeout,
            env=env,
            check=False,
        )
        return result.returncode == 0, result.stdout, result.stderr
    except subprocess.TimeoutExpired:
        return False, '', f"Command timed out after {timeout}s: {' '.join(args)}"
    except Exception as e:
        return False, '', str(e)


def update_repository():
    """Update /home/pi/RaspberryPi-CAN to exactly match origin/master."""
    repo_path = os.path.expanduser('~/RaspberryPi-CAN')

    print(f"[{datetime.now()}] Starting repository update...")

    if not os.path.exists(repo_path):
        print(f"ERROR: Repository directory {repo_path} does not exist")
        return False
    print(f"Repository directory found: {repo_path}")

    print('Checking internet connection...')
    if not check_internet_connection():
        print('ERROR: No internet connection available. Cannot update repository.')
        print('Please connect to the internet and try again.')
        return False
    print('Internet connection available')

    success, stdout, stderr = run_command(['git', 'rev-parse', '--is-inside-work-tree'], cwd=repo_path)
    if not success or stdout.strip() != 'true':
        print(f"ERROR: {repo_path} is not a git repository")
        print(stderr.strip())
        return False
    print('Git repository confirmed')

    print('Fetching updates from origin...')
    success, stdout, stderr = run_command(['git', 'fetch', 'origin'], cwd=repo_path, timeout=180)
    if not success:
        print('ERROR: Failed to fetch from origin:')
        print(stderr.strip())
        return False
    print('Fetched from origin')

    print('Discarding local tracked changes...')
    success, stdout, stderr = run_command(['git', 'reset', '--hard'], cwd=repo_path)
    if not success:
        print('ERROR: Failed to discard local changes:')
        print(stderr.strip())
        return False
    print('Tracked changes discarded')

    print('Cleaning local untracked and ignored files...')
    success, stdout, stderr = run_command(['git', 'clean', '-fdx'], cwd=repo_path)
    if not success:
        print('ERROR: Failed to clean local files:')
        print(stderr.strip())
        return False
    if stdout.strip():
        print(stdout.strip())
    print('Local files cleaned')

    print('Checking out master from origin/master...')
    success, stdout, stderr = run_command(['git', 'checkout', '-B', 'master', 'origin/master'], cwd=repo_path)
    if not success:
        print('ERROR: Failed to check out master:')
        print(stderr.strip())
        return False
    print('Checked out master')

    print('Resetting to origin/master...')
    success, stdout, stderr = run_command(['git', 'reset', '--hard', 'origin/master'], cwd=repo_path)
    if not success:
        print('ERROR: Failed to reset to origin/master:')
        print(stderr.strip())
        return False
    print('Reset complete')

    print('Final cleanup...')
    success, stdout, stderr = run_command(['git', 'clean', '-fdx'], cwd=repo_path)
    if not success:
        print('ERROR: Final cleanup failed:')
        print(stderr.strip())
        return False
    if stdout.strip():
        print(stdout.strip())

    success, stdout, stderr = run_command(['git', 'log', '-1', '--oneline'], cwd=repo_path)
    if success and stdout.strip():
        print(f"Current commit: {stdout.strip()}")

    success, stdout, stderr = run_command(['git', 'status', '--short'], cwd=repo_path)
    if success and stdout.strip():
        print('ERROR: Working tree is not clean:')
        print(stdout.strip())
        return False

    print(f"[{datetime.now()}] Repository update completed successfully")
    return True


try:
    success = update_repository()
    print('')
    if success:
        print('Repository update completed successfully')
    else:
        print('Repository update failed. Check the error messages above.')
except KeyboardInterrupt:
    print('')
    print('Update cancelled by user')
except Exception as e:
    print('')
    print(f"Unexpected error occurred: {e}")